In [71]:
import pandas as pd;
import openpyxl
from openpyxl.styles import NamedStyle


In [72]:
#Lê a aba de Configurações do Imóvel
df_conf_imovel = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='Configuração-Imóvel')

#Remove colunas indesejadas
df_filtrado = df_conf_imovel.drop(columns=['neighborhood', 'state', 'status'])

# Converte as colunas da aba para dicionário com chaves em letras minúsculas
code_moveis_comodos = [{k.lower(): v for k, v in item.items()} for item in df_filtrado.to_dict(orient='records')]

print(code_moveis_comodos)

[{'code': 'OKA101', 'city': 'Florianópolis', 'camas de solteiro': 0, 'cama de casal': 1, 'cama queen': 0, 'cama king': 0, 'sofa cama solteiro': 1, 'sofa cama casal': 0, 'travesseiros': 4, 'numero quartos': 1, 'numero banheiros': 1, 'numero lavabos': 0, 'numero hóspedes': 3}, {'code': 'OKA102', 'city': 'Florianópolis', 'camas de solteiro': 1, 'cama de casal': 1, 'cama queen': 0, 'cama king': 0, 'sofa cama solteiro': 0, 'sofa cama casal': 0, 'travesseiros': 4, 'numero quartos': 1, 'numero banheiros': 1, 'numero lavabos': 0, 'numero hóspedes': 3}, {'code': 'OKA105', 'city': 'Florianópolis', 'camas de solteiro': 0, 'cama de casal': 1, 'cama queen': 0, 'cama king': 0, 'sofa cama solteiro': 1, 'sofa cama casal': 0, 'travesseiros': 4, 'numero quartos': 1, 'numero banheiros': 1, 'numero lavabos': 0, 'numero hóspedes': 3}, {'code': 'OKA106', 'city': 'Florianópolis', 'camas de solteiro': 0, 'cama de casal': 0, 'cama queen': 1, 'cama king': 0, 'sofa cama solteiro': 1, 'sofa cama casal': 0, 'trave

In [73]:
def pega_total_amenities(num_hospedes):
    df_amenities = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='BD - Amenities')
    total_amenities = round(float(df_amenities['total final'].sum()),2)
    return total_amenities * num_hospedes

In [74]:
#Pega o número de imóveis da lista de dicionários
num_imoveis = len(code_moveis_comodos)

#Percorre os imóveis armazenados no dicionario
for imovel in range(num_imoveis):
    num_hospedes = float(code_moveis_comodos[imovel]['numero hóspedes'])
    total_amenities = pega_total_amenities(num_hospedes)
    code_moveis_comodos[imovel]['total de gastos de amenities'] = total_amenities

In [85]:
df_estoque_imovel = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='Estoque-Imóvel')

df_filtrado = df_estoque_imovel.drop(columns=['neighborhood', 'state', 'status', 'city']) #remove colunas indesejadas
code_imovel_estoque = df_filtrado.to_dict(orient='records') #Faz um dicionário que armazena o codigo do imóvel e seu estoque
estoque_do_imovel = len(code_imovel_estoque)
comodos_do_imovel = len(code_moveis_comodos)

#unifica abas 'Configuração do imóvel' e 'Estoque do imóvel'
for comodos in range(comodos_do_imovel): 
    chave_code = code_moveis_comodos[comodos]['code'] #pega o codigo do imovel 

    for imovel in range(estoque_do_imovel): #percorre a lista relacionada ao estoque dos imoveis
        code = code_imovel_estoque[imovel]['code'] 
        
        if code == chave_code: #se o codigo existe em ambos os dicionarios...
            for k,v in code_imovel_estoque[imovel].items():
                code_moveis_comodos[imovel][k.lower()] = v #adiciona as infos de estoque do imovel no dicionario de comodos
            break
        
print(code_moveis_comodos)

[{'code': 'OKA101', 'city': 'Florianópolis', 'camas de solteiro': 0, 'cama de casal': 1, 'cama queen': 0, 'cama king': 0, 'sofa cama solteiro': 1, 'sofa cama casal': 0, 'travesseiros': 4, 'numero quartos': 1, 'numero banheiros': 1, 'numero lavabos': 0, 'numero hóspedes': 3, 'total de gastos de amenities': 9.84, 'toalha de rosto': 2, 'toalha de piso': 2, 'toalha de maquiagem': 0, 'toalha de banho': 7, 'pano de prato': 0, 'pano de pia': 0, 'manta queen size': 1, 'lençol solteiro': 2, 'lençol queen size': 0, 'lençol king size': 0, 'lençol casal': 6, 'fronha': 7, 'edredom solteiro': 0, 'edredom queen size': 0, 'edredom king size': 0, 'edredom casal': 3, 'colcha solteiro': 0, 'colcha queen size': 0, 'colcha casal': 0, 'cobertor queen size': 0, 'capa edredom solteiro': 0, 'capa edredom queen size': 0, 'capa edredom king size': 0, 'capa edredom casal': 3}, {'code': 'OKA102', 'city': 'Florianópolis', 'camas de solteiro': 1, 'cama de casal': 1, 'cama queen': 0, 'cama king': 0, 'sofa cama soltei

In [86]:
def soma_elementos_lista(lista):
    soma = 0
    for numero in lista:
        soma += numero
    return soma

In [97]:
#Encontra o custo da lavanderia de cada imóvel

df_valor_lavanderia = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='BD - Valor Lavanderia Por Item') 
colunas_de_interesse = []

for coluna in df_valor_lavanderia.columns:
    if (coluna.lower() != 'city' and coluna.lower() != 'média original por cidade (m) ' and coluna.lower() != 'unnamed: 25'):
        colunas_de_interesse.append(coluna)

qtde_imoveis = len(code_moveis_comodos) 
for imovel in range(qtde_imoveis): 
    dados_do_imovel = code_moveis_comodos[imovel] #pega dicionario com infos do imovel
    cidade_do_imovel = dados_do_imovel['city']
    df_filtro_cidade = df_valor_lavanderia[df_valor_lavanderia['City'] == cidade_do_imovel]
    df_apenas_valores = df_filtro_cidade[colunas_de_interesse]  # Filtra as colunas de interesse na planilha
    preco_lavanderia_imovel = []

    for produto in colunas_de_interesse:
        nome_do_produto = produto.lower()
        qtde_produto_imovel = dados_do_imovel.get(nome_do_produto, 0)  # Usa get para evitar erro se a chave não existir
        valor_produto = round(df_apenas_valores[produto].values[0], 2)  # Arredonda para 2 casas decimais
        preco_lavanderia_imovel.append(qtde_produto_imovel * valor_produto)
    code_moveis_comodos[imovel]['total de gastos da lavanderia'] = soma_elementos_lista(preco_lavanderia_imovel)


['Toalha de Maquiagem', 'Pano de Pia', 'Fronha', 'Toalha de rosto', 'Pano de Prato', 'Toalha de piso', 'Toalha de banho', 'Lençol solteiro', 'Lençol casal', 'Lençol queen size', 'Lençol king size', 'Manta queen size', 'Edredom solteiro', 'Edredom casal', 'Edredom queen size', 'Edredom king size', 'Capa edredom solteiro', 'Capa edredom casal', 'Capa edredom queen size', 'Capa edredom king size', 'Colcha solteiro', 'Colcha casal', 'Colcha queen size', 'Cobertor queen size']


In [ ]:
#Encontrar o custo da lavanderia de cada imóvel

qtde_imoveis = len(code_moveis_comodos) 
df_valor_lavanderia = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='BD - Valor Lavanderia Por Item') 
for imovel in range(qtde_imoveis): 
    dados_do_imovel = code_moveis_comodos[imovel] #pega dicionario com infos do imovel
    cidade_do_imovel = dados_do_imovel['city']
    df_filtro_cidade = df_apenas_valores[df_valor_lavanderia['City'] == cidade_do_imovel]
    preco_lavanderia_imovel = []
    print(f"\nImóvel: {code_moveis_comodos[imovel]['code']}\nProdutos:")
    for produto in df_filtro_cidade.columns:
        nome_do_produto = produto.lower()
        if (nome_do_produto != 'Média Original por cidade (m)' and nome_do_produto !='City'):
            qtde_produto_imovel = dados_do_imovel.get(nome_do_produto, 0)  # Usa get para evitar erro se a chave não existir
            print(f"-> {nome_do_produto} - quantidade: {qtde_produto_imovel}")
            valor_produto = round(float(df_apenas_valores[produto].values[0]), 2)
            if pd.notna(valor_produto):
                    preco_lavanderia_imovel.append(qtde_produto_imovel * valor_produto)
    code_moveis_comodos[imovel]['total de gastos da lavanderia'] = soma_elementos_lista(preco_lavanderia_imovel)


In [59]:
#Limpa o dicionário pra deixar apenas os custos finais (apaga quantidade de comodos e itens do dicionario)
qtde_imoveis = len(code_moveis_comodos) 
lista_de_imoveis = []
for imovel in range(qtde_imoveis):
    dados_do_imovel = code_moveis_comodos[imovel]
    dic_auxiliar = {}
    for item,quantidade in dados_do_imovel.items():
        if (item == 'code' or item == 'city' or item == 'total de gastos da lavanderia' or item == 'total de gastos de amenities' or item == 'numero banheiros' or item == 'numero quartos'):
            dic_auxiliar[item] = quantidade
    lista_de_imoveis.append(dic_auxiliar)
code_moveis_comodos.clear()
code_moveis_comodos = lista_de_imoveis   
print(code_moveis_comodos) 

[{'code': 'OKA101', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 260.06}, {'code': 'OKA102', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 166.1}, {'code': 'OKA105', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 195.07}, {'code': 'OKA106', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 283.81}, {'code': 'OKA110', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 268.56}, {'code': 'OKA111', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 1

In [60]:
#Encontrando o custo do material de limpeza por imóvel:

df_material_limpeza = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='BD - Material de Limpeza')
df_material_limpeza = df_material_limpeza.drop(columns=['valor'])

#transforma a aba em uma lista de dicionários (cada dic corresponde a uma coluna da tabela)
itens_de_limpeza = [{k.lower(): v for k, v in item.items()} for item in df_material_limpeza.to_dict(orient='records')]
total_geral = itens_de_limpeza[-1] #pega o ultimo dic (que armazena o somatorio dos custos da aba)
print(total_geral)
qtde_imoveis = len(code_moveis_comodos) 
for imovel in range(qtde_imoveis):
    dados_do_imovel = code_moveis_comodos[imovel]
    custo_material_limpeza = dados_do_imovel['numero banheiros'] * total_geral['total']
    code_moveis_comodos[imovel]['total de gastos de material de limpeza'] = custo_material_limpeza
    print(code_moveis_comodos[imovel])


{'item': nan, 'quantidade por imóvel': 'Total geral:', 'total': 46.094}
{'code': 'OKA101', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 260.06, 'total de gastos de material de limpeza': 46.094}
{'code': 'OKA102', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 166.1, 'total de gastos de material de limpeza': 46.094}
{'code': 'OKA105', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 195.07, 'total de gastos de material de limpeza': 46.094}
{'code': 'OKA106', 'city': 'Florianópolis', 'numero quartos': 1, 'numero banheiros': 1, 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 283.81, 'total de gastos de material de limpeza': 46.094}
{'code': 'OKA110', 'city': 'Florianópolis', 'numero quartos':

In [61]:
def preco_limpeza_quartos(df_limpeza_diarista, num_quartos):
    linha_nro_quartos= df_limpeza_diarista[df_limpeza_diarista['Numero de Quartos'] == num_quartos]
    return linha_nro_quartos['Valor'].values[0]


In [62]:
#Encontrando o custo da limpeza do(a) diarista e armazenando no dicionário de dados do imóvel:

qtde_imoveis = len(code_moveis_comodos) 
df_limpeza_diarista = pd.read_excel('Parametros - Taxa de Limpeza.xlsx', sheet_name='BD - Valor Limpeza Diarista')
for imovel in range(qtde_imoveis):
    valor_limpeza = preco_limpeza_quartos(df_limpeza_diarista, code_moveis_comodos[imovel]['numero quartos'])
    code_moveis_comodos[imovel]['total de gastos da limpeza diarista'] = valor_limpeza

In [ ]:
#Encontrando a taxa de limpeza final:
qtde_imoveis = len(code_moveis_comodos) 
for imovel in range(qtde_imoveis):
    custos = code_moveis_comodos[imovel]
    taxa_de_limpeza = round(float(custos['total de gastos da limpeza diarista'] + custos['total de gastos de material de limpeza'] + custos['total de gastos da lavanderia'] + custos['total de gastos de amenities']),2)

    #armazena o valor da taxa de limpeza no dicionario de dados do imovel:
    code_moveis_comodos[imovel]['taxa de limpeza'] = taxa_de_limpeza

    #remove colunas que agora são desnecessárias:
    code_moveis_comodos[imovel].pop('city')
    code_moveis_comodos[imovel].pop('numero quartos')
    code_moveis_comodos[imovel].pop('numero banheiros')    

    print(code_moveis_comodos[imovel])
    print()

In [69]:
# Converter a lista de dicionários para um DataFrame
print(code_moveis_comodos)
df_taxa_limpeza = pd.DataFrame(code_moveis_comodos)

# Carregar o arquivo Excel existente
file_path = 'Parametros - Taxa de Limpeza.xlsx'
workbook = openpyxl.load_workbook(file_path)

# Selecionar a aba específica que deseja modificar
sheet = workbook['TaxaLimpeza (Atual)']


# Atualizar os dados da aba selecionada
for row_idx, data_row in enumerate(df_taxa_limpeza.itertuples(index=False), start=2):  # Começa na linha 2 para pular o cabeçalho
    for col_idx, value in enumerate(data_row, start=1):  # Começa na coluna 1 (A)
        cell = sheet.cell(row=row_idx, column=col_idx)
        cell.value = value

# Criar um estilo de célula para moeda em Real brasileiro
currency_style = NamedStyle(name='currency_style', number_format='R$ #,##0.00')

# Definir as colunas que você deseja formatar como Moeda
# Ajuste as colunas conforme a sua planilha
currency_columns = [2, 3, 4, 5, 6, 7]  # Atualize conforme as colunas de valores na sua planilha

for col in currency_columns:
    for row in range(2, sheet.max_row + 1):  # Assume que a primeira linha é o cabeçalho
        cell = sheet.cell(row=row, column=col)
        cell.style = currency_style

# Salvar o arquivo Excel com as atualizações
workbook.save('Resultado - Taxa de Limpeza.xlsx')

[{'code': 'OKA101', 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 260.06, 'total de gastos de material de limpeza': 46.094, 'total de gastos da limpeza diarista': np.int64(120), 'taxa de limpeza': 435.99}, {'code': 'OKA102', 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 166.1, 'total de gastos de material de limpeza': 46.094, 'total de gastos da limpeza diarista': np.int64(120), 'taxa de limpeza': 342.03}, {'code': 'OKA105', 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 195.07, 'total de gastos de material de limpeza': 46.094, 'total de gastos da limpeza diarista': np.int64(120), 'taxa de limpeza': 371.0}, {'code': 'OKA106', 'total de gastos de amenities': 9.84, 'total de gastos da lavanderia': 283.81, 'total de gastos de material de limpeza': 46.094, 'total de gastos da limpeza diarista': np.int64(120), 'taxa de limpeza': 459.74}, {'code': 'OKA110', 'total de gastos de amenities': 9.84, 'total de gastos da lava